In [ ]:
import gc
import random
from pathlib import Path
from tempfile import gettempdir

import matplotlib as mpl
import matplotlib.pyplot as pyplot
import numpy as np
import pandas as pd
import scipy as sp
import scipy.constants
from IPython.core.display import display
from matplotlib import cm, ticker
from numpy import ma

In [ ]:
## Adjust plot defaults
mpl.rcParams["font.sans-serif"] = ["DejaVu Sans", "sans-serif"]

# Toggle on/off interactive notebooks
if False:
    %matplotlib notebook
    FIGSIZE = 6
else:
    %matplotlib inline
    FIGSIZE = 12
    
mpl.rcParams["figure.figsize"] = [FIGSIZE, FIGSIZE / sp.constants.golden]

In [ ]:
def random_drop(df, count=100_000, inplace=False):
    ratio = count / df.index.size
    selector = list(filter(lambda _: random.random() > ratio, df.index))
    return df.drop(selector, inplace=inplace)

In [ ]:
# OUTPUT_DIR = Path(gettempdir()) / "leptogenesis"
OUTPUT_DIR = Path("../output/") / "leptogenesis"
data = dict()

# Number Density Solutions

In [ ]:
data["n"] = random_drop(pd.read_csv(OUTPUT_DIR / "n.csv"))
# data["n"] = random_drop(pd.read_csv(OUTPUT_DIR / "n.csv").query("1e-8 <= beta <= 1e-5"))
XSCALE = "log" if data["n"]["beta"].max() / data["n"]["beta"].min() > 100 else "linear"

gc.collect()

In [ ]:
fig, ax = pyplot.subplots()
ax.set_title("Integration Evolution")
ax.set_yscale("log")
data["n"].plot(x="step", y="beta", ax=ax)

pyplot.show()

In [ ]:
fig, axs = pyplot.subplots(
    nrows=3, sharex=True, figsize=[FIGSIZE, 3 * FIGSIZE / sp.constants.golden_ratio]
)
for ax in axs:
    ax.set_xscale(XSCALE)

## B-L
axs[0].set_title("B-L")
data["n"].abs().plot(x="beta", y=data["n"].columns[2:3], ax=axs[0])
axs[0].set_yscale("log")
axs[0].set_ylim(bottom=1e-20)

## Number density
axs[1].set_title("Number Densities")
data["n"].plot(
    x="beta",
    y=data["n"].columns[5::3],
    ax=axs[1],
    color=mpl.rcParams["axes.prop_cycle"].by_key()["color"],
)
data["n"].plot(
    x="beta",
    y=data["n"].columns[6::3],
    ax=axs[1],
    color=mpl.rcParams["axes.prop_cycle"].by_key()["color"],
    linestyle="dashed",
)
axs[1].set_ylim(bottom=-0.5, top=5)
# axs[1].set_yscale("log")
# axs[1].set_ylim(bottom=1e-20)

## Change in Number Density
axs[2].set_title("Change in Number Densities")
data["n"].plot(
    x="beta", 
    y=data["n"].columns[7::3], 
    ax=axs[2],
    color=mpl.rcParams["axes.prop_cycle"].by_key()["color"],
)
data["n"].apply(lambda c: c.apply(np.negative) if c.name not in ["step", "beta"] else c).plot(
    x="beta", 
    y=data["n"].columns[7::3], 
    ax=axs[2],
    color=mpl.rcParams["axes.prop_cycle"].by_key()["color"],
    linestyle="dashed",
)
axs[2].set_yscale("log")
axs[2].set_ylim(bottom=1e-20)

pyplot.show()

# Interactions

## $H \leftrightarrow -L_{i_2} N_{i_3}$

In [ ]:
data["HLN"] = random_drop(
    pd.read_csv(OUTPUT_DIR / "n_el_h.csv")
    .query("{} <= beta <= {}".format(data["n"]["beta"].min(), data["n"]["beta"].max()))
    .sort_values(by=["beta", "step"])
)

gc.collect()

In [ ]:
fig, ax = pyplot.subplots(
    nrows=5, sharex=True, figsize=[FIGSIZE, 5 * FIGSIZE / sp.constants.golden_ratio]
)

for offset, title in enumerate(
    ["Decaying Particle Index", "Gamma", "Decay", "Inverse Decay", "Net Decay"]
):
    ax[offset].set_title(title)
    ax[offset].set_xscale(XSCALE)
    if title in ["Gamma", "Net Decay"]:
        ax[offset].set_yscale("log")
        data["HLN"].abs().plot(
            x="beta", y=data["HLN"].columns[2 + offset :: 3], ax=ax[offset]
        )
    else:
        # ax.set_yscale("symlog")
        data["HLN"].plot(
            x="beta", y=data["HLN"].columns[2 + offset :: 3], ax=ax[offset]
        )

pyplot.show()

# Scan

In [ ]:
data["scan"] = pd.read_csv(OUTPUT_DIR / "scan.csv").sort_values(by=["y", "m"])

gc.collect()

In [ ]:
x = data["scan"]["y"].unique()
y = data["scan"]["m"].unique()
z = data["scan"]["B-L"].abs().values.reshape(x.size, y.size)
# z = ma.masked_where(np.isnan(z), z)

fig, ax = pyplot.subplots()

ax.set_xlabel("$y_v$")
ax.set_xscale("log")
ax.set_ylabel("$N_1$ mass [GeV]")
ax.set_yscale("log")
# cs = ax.contourf(x, y, z, locator=ticker.LogLocator(subs=None))
cs = ax.contourf(x, y, z)
ax.plot(data["scan"]["y"], data["scan"]["m"], "ko", ms=3)
cbar = fig.colorbar(cs, ticks=ticker.LogLocator())
cbar.set_label("B-L")

pyplot.show()